# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 22 2022 9:01AM,240152,10,Enova-10185,Emerginnova,Released
1,Jun 22 2022 9:01AM,240152,10,Enova-10186,Emerginnova,Released
2,Jun 22 2022 8:48AM,240149,10,SO94297,"Senseonics, Incorporated",Released
3,Jun 22 2022 8:46AM,240148,16,8153607,Sartorius Lab Products and Service,Executing
4,Jun 22 2022 8:31AM,240147,19,TR-RA-06232022,"GCH Granules USA, Inc.",Released
5,Jun 22 2022 8:30AM,240134,10,CTF0011261,"Citieffe, Inc.",Released
6,Jun 22 2022 8:30AM,240130,10,0085788720,ISDIN Corporation,Released
7,Jun 22 2022 8:30AM,240130,10,0085788735,ISDIN Corporation,Released
8,Jun 22 2022 8:30AM,240130,10,0085788739,ISDIN Corporation,Released
9,Jun 22 2022 8:30AM,240130,10,0085788740,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,240146,Released,40
39,240147,Released,1
40,240148,Executing,1
41,240149,Released,1
42,240152,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240146,NaN,NaN,40.0
240147,NaN,NaN,1.0
240148,NaN,1.0,NaN
240149,NaN,NaN,1.0
240152,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240030,0.0,0.0,1.0
240034,0.0,0.0,9.0
240061,0.0,1.0,0.0
240079,0.0,3.0,9.0
240081,0.0,35.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240030,0,0,1
240034,0,0,9
240061,0,1,0
240079,0,3,9
240081,0,35,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240030,0,0,1
1,240034,0,0,9
2,240061,0,1,0
3,240079,0,3,9
4,240081,0,35,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240030,,,1
1,240034,,,9
2,240061,,1,
3,240079,,3,9
4,240081,,35,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 22 2022 9:01AM,240152,10,Emerginnova
2,Jun 22 2022 8:48AM,240149,10,"Senseonics, Incorporated"
3,Jun 22 2022 8:46AM,240148,16,Sartorius Lab Products and Service
4,Jun 22 2022 8:31AM,240147,19,"GCH Granules USA, Inc."
5,Jun 22 2022 8:30AM,240134,10,"Citieffe, Inc."
6,Jun 22 2022 8:30AM,240130,10,ISDIN Corporation
12,Jun 22 2022 8:07AM,240146,10,ISDIN Corporation
52,Jun 22 2022 8:02AM,240145,19,"AdvaGen Pharma, Ltd"
53,Jun 22 2022 7:55AM,240144,10,ISDIN Corporation
84,Jun 22 2022 7:50AM,240143,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 22 2022 9:01AM,240152,10,Emerginnova,,,2
1,Jun 22 2022 8:48AM,240149,10,"Senseonics, Incorporated",,,1
2,Jun 22 2022 8:46AM,240148,16,Sartorius Lab Products and Service,,1,
3,Jun 22 2022 8:31AM,240147,19,"GCH Granules USA, Inc.",,,1
4,Jun 22 2022 8:30AM,240134,10,"Citieffe, Inc.",,,1
5,Jun 22 2022 8:30AM,240130,10,ISDIN Corporation,,1,5
6,Jun 22 2022 8:07AM,240146,10,ISDIN Corporation,,,40
7,Jun 22 2022 8:02AM,240145,19,"AdvaGen Pharma, Ltd",,,1
8,Jun 22 2022 7:55AM,240144,10,ISDIN Corporation,,,31
9,Jun 22 2022 7:50AM,240143,19,"AdvaGen Pharma, Ltd",,,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 9:01AM,240152,10,Emerginnova,2,,
1,Jun 22 2022 8:48AM,240149,10,"Senseonics, Incorporated",1,,
2,Jun 22 2022 8:46AM,240148,16,Sartorius Lab Products and Service,,1,
3,Jun 22 2022 8:31AM,240147,19,"GCH Granules USA, Inc.",1,,
4,Jun 22 2022 8:30AM,240134,10,"Citieffe, Inc.",1,,
5,Jun 22 2022 8:30AM,240130,10,ISDIN Corporation,5,1,
6,Jun 22 2022 8:07AM,240146,10,ISDIN Corporation,40,,
7,Jun 22 2022 8:02AM,240145,19,"AdvaGen Pharma, Ltd",1,,
8,Jun 22 2022 7:55AM,240144,10,ISDIN Corporation,31,,
9,Jun 22 2022 7:50AM,240143,19,"AdvaGen Pharma, Ltd",9,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 9:01AM,240152,10,Emerginnova,2,,
1,Jun 22 2022 8:48AM,240149,10,"Senseonics, Incorporated",1,,
2,Jun 22 2022 8:46AM,240148,16,Sartorius Lab Products and Service,,1,
3,Jun 22 2022 8:31AM,240147,19,"GCH Granules USA, Inc.",1,,
4,Jun 22 2022 8:30AM,240134,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 9:01AM,240152,10,Emerginnova,2.0,NaN,NaN
1,Jun 22 2022 8:48AM,240149,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Jun 22 2022 8:46AM,240148,16,Sartorius Lab Products and Service,NaN,1.0,NaN
3,Jun 22 2022 8:31AM,240147,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,Jun 22 2022 8:30AM,240134,10,"Citieffe, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 9:01AM,240152,10,Emerginnova,2.0,0.0,0.0
1,Jun 22 2022 8:48AM,240149,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Jun 22 2022 8:46AM,240148,16,Sartorius Lab Products and Service,0.0,1.0,0.0
3,Jun 22 2022 8:31AM,240147,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,Jun 22 2022 8:30AM,240134,10,"Citieffe, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3121474,133.0,5.0,26.0
102,240089,1.0,0.0,0.0
15,480270,70.0,0.0,0.0
16,720389,5.0,3.0,0.0
19,2161046,21.0,15.0,3.0
20,960388,23.0,54.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3121474,133.0,5.0,26.0
1,102,240089,1.0,0.0,0.0
2,15,480270,70.0,0.0,0.0
3,16,720389,5.0,3.0,0.0
4,19,2161046,21.0,15.0,3.0
5,20,960388,23.0,54.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,133.0,5.0,26.0
1,102,1.0,0.0,0.0
2,15,70.0,0.0,0.0
3,16,5.0,3.0,0.0
4,19,21.0,15.0,3.0
5,20,23.0,54.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,133.0
1,102,Released,1.0
2,15,Released,70.0
3,16,Released,5.0
4,19,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,19,20
Status,,,,,,
Completed,26.0,0.0,0.0,0.0,3.0,0.0
Executing,5.0,0.0,0.0,3.0,15.0,54.0
Released,133.0,1.0,70.0,5.0,21.0,23.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,19,20
0,Completed,26.0,0.0,0.0,0.0,3.0,0.0
1,Executing,5.0,0.0,0.0,3.0,15.0,54.0
2,Released,133.0,1.0,70.0,5.0,21.0,23.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,19,20
0,Completed,26.0,0.0,0.0,0.0,3.0,0.0
1,Executing,5.0,0.0,0.0,3.0,15.0,54.0
2,Released,133.0,1.0,70.0,5.0,21.0,23.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()